Практическое задание №8 Абдуллаева Вера группа 494а

1) Необходимо выбрать любой (интересный на ваш взгляд) набор данных, подходящий для решения задачи классификации (https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets, например diabet, wine, breast_cancer; https://archive.ics.uci.edu/ml/index.php - выбор большой, kaggle.com)

2) Выбрать не менее 3-х алгоритмов классификации данных, обосновать выбор, обучить модели. 

3) Оценить качество классификации. Подход к оценке и метрики оценки выбрать самостоятельно, обосновать выбор.

4) Проинтерпретировать полученный результат.

Будем использовать данные о клиентах страховой фирмы.

Опишем ниже кратко, что было предпринято для классификации и обучений моделей.

Почему был испсользован классификатор RandomForestClassifier():

Дерево решений — интуитивно понятная базовая единица алгоритма случайный лес. Мы можем рассматривать его как серию вопросов да/нет о входных данных. В конечном итоге вопросы приводят к предсказанию определённого класса (или величины в случае регрессии). Это интерпретируемая модель, так как решения принимаются так же, как и человеком: мы задаём вопросы о доступных данных до тех пор, пока не приходим к определённому решению (в идеальном мире).

Базовая идея дерева решений заключается в формировании запросов, с которыми алгоритм обращается к данным. При использовании алгоритма CART вопросы (также называемые разделением узлов) определяются таким образом, чтобы ответы вели к уменьшению загрязнения Джини (Gini Impurity). Это означает, что дерево решений формирует узлы, содержащие большое количество образцов (из набора исходных данных), принадлежащих к одному классу. Алгоритм старается обнаружить параметры со сходными значениями.

Немного о LogisticRegression или логистическая регрессия, которая является еще одним классификатором. Основным его свойством является
тот факт, что он корректно оценивает вероятность принадлежности объекта к каждому из классов.


GridSearchCV – это очень мощный инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров.
Данный метод хоть и работает не очень быстро, но, при этом, экономит достаточно времени по сравнению с ручным перебором тех же параметров, чем дает явное преимущество в использовании при построении моделей.

KNeighborsClassifier классификатор

На интуитивном уровне суть метода проста: посмотри на соседей вокруг, какие из них преобладают, таковым ты и являешься. Формально основой метода является гипотеза компактности: если метрика расстояния между примерами введена удачно, то схожие примеры гораздо чаще лежат в одном классе, чем в разных.
В случае использования метода для классификации объект присваивается тому классу, который является наиболее распространённым среди k соседей данного элемента, классы которых уже известны.
В случае использования метода для регрессии, объекту присваивается среднее значение по k ближайшим к нему объектам, значения которых уже известны.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

data = pd.read_csv('C:\\Users\\user\\Desktop\\Методы анализа и обработки данных\\moad_lab8\\insurance.csv')
data.head()
#Для создания модели нам необходимо вытащить целевой признак «smoker» в переменную y, а оставшиеся признаки в переменную X, предварительно
# удалив неинтересующие нас при классификации данные:
X = data.drop(['bmi','region','charges','smoker'], axis = 1)

# Устанавливаем новые значения данных пола для работы с числовыми значениями
X['sex'].where(~(X.sex =='male'), other=float(0), inplace=True)
X['sex'].where(~(X.sex =='female'), other=float(1), inplace=True)

y = data[['smoker']]
#Разделяем наши данные для обучения
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size = 0.8, random_state = 1)

#Определяем модели, которые будем обучать (способы классификации данных)
logregr_cv = LogisticRegression(random_state = 0)
randforest_cv = RandomForestClassifier()
kn_cv = KNeighborsClassifier()


cv_dict = {0: 'Логистическая регрессия', 
           1: 'Случайный лес', 
           2: 'Метод k ближайших соседей'}


cv_models = [logregr_cv, randforest_cv, kn_cv]
           
for i,model in enumerate(cv_models):
    print("{} | Тестовая точность измерений: {}".format(cv_dict[i], 
           cross_val_score(model, X, y.values.ravel(), cv = 10, scoring = 'accuracy').mean()))



clsf = RandomForestClassifier()
parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }
grid = GridSearchCV(clsf, parametrs, cv=3) 
#CV – перекрёстная проверка (кросс-валидация, Cross-validation), метод, который показывает, что модель не переобучилась.
# Если все результаты (в данном случае обучение проходит 3х моделей по 3м разным выборкам) хороши, значит модель не переобучена.
best_model = grid.fit(X_train, y_train.values.ravel())
Test = np.array([[40,0,0]]) 
# Возраст\Пол\Количество детей
#Предсказываем курит ли человек
prediction = best_model.predict(Test)
print(prediction)


Логистическая регрессия | Тестовая точность измерений: 0.7952193917629895
Случайный лес | Тестовая точность измерений: 0.7376725395578498
Метод k ближайших соседей | Тестовая точность измерений: 0.7600998765570643
['no']
